# Convert SQLite output(s) to parquet files with CytoTable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Metadata_ZSlice, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../3.feature_extraction/sqlite_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plate(s) in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 1 plate(s) in this dataset. Below are the names:
NF0014


## Convert SQLite to parquet file(s) for single-cell profiles

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/converted_profiles/{file_path.stem}_sc_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=500
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: NF0014
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])
    
    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells",
        ]
        + [col for col in df.columns if col not in [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells"]]
    ].rename(
        columns=lambda col: "Metadata_" + col if col in [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells"
        ] else col
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet(
    f"./data/converted_profiles/{plate_names[0]}_sc_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(2142, 2910)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Image_Metadata_ZSlice,...,Nuclei_Texture_Variance_ER_3_02_256,Nuclei_Texture_Variance_ER_3_03_256,Nuclei_Texture_Variance_Hoechst_3_00_256,Nuclei_Texture_Variance_Hoechst_3_01_256,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mito_3_00_256,Nuclei_Texture_Variance_Mito_3_01_256,Nuclei_Texture_Variance_Mito_3_02_256,Nuclei_Texture_Variance_Mito_3_03_256
0,881.879955,915.249151,897.356882,897.287549,1,45,NF0014,2,E10,ZS012,...,8.812134,8.467408,108.839056,107.088383,110.573107,104.592520,6.244812,6.511083,6.112579,6.104619
1,582.367521,1021.031674,583.273637,1065.691171,7,1,NF0014,1,C10,ZS044,...,3.085660,3.016230,53.810984,53.505842,52.039812,49.243901,109.565486,109.867762,109.500629,112.251717
2,543.574542,916.394750,570.123399,929.378641,9,2,NF0014,2,C10,ZS018,...,1.461975,1.549177,1186.463306,1215.197101,1182.212837,1157.999266,12.582731,12.280363,12.860938,13.047288
3,468.025680,679.002266,473.275746,734.014534,16,3,NF0014,1,C11,ZS030,...,1.184913,1.206024,1516.607560,1472.995128,1531.257012,1522.003404,10.054106,9.948455,10.055683,10.099761
4,1130.959659,811.376705,1140.627994,775.280979,18,6,NF0014,2,C2,ZS006,...,371.230006,374.107620,631.486059,624.970380,613.398910,606.612988,82.094469,82.249120,82.790418,83.836445


## Extract organoid only profiles

In [6]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/converted_profiles/{file_path.stem}_organoid_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)

    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        metadata=["image"],
        compartments=["organoids"],
        identifying_columns=["ImageNumber"],
        joins="""
        SELECT
            *
        FROM
            read_parquet('per_image.parquet') as per_image
        INNER JOIN read_parquet('per_organoids.parquet') AS per_organoids ON
            per_organoids.Metadata_ImageNumber = per_image.Metadata_ImageNumber
        """,
        page_keys={
            "image": "ImageNumber",
            "organoids": "Organoids_Number_Object_Number",
            "join": "Organoids_Number_Object_Number",
        },
        chunk_size=10,
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: NF0014
All plates have been converted with cytotable!


In [7]:
converted_df = pd.read_parquet(
    f"./data/converted_profiles/{plate_names[0]}_organoid_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(152, 1499)


,Metadata_ImageNumber,Image_ExecutionTime_02Metadata,Image_Metadata_Channel,Image_Metadata_FileLocation,Image_Metadata_Frame,Image_Metadata_Plate,Image_Metadata_Series,Image_Metadata_Site,Image_Metadata_Well,Image_Metadata_ZSlice,...,Organoids_Texture_Variance_ER_3_02_256,Organoids_Texture_Variance_ER_3_03_256,Organoids_Texture_Variance_Hoechst_3_00_256,Organoids_Texture_Variance_Hoechst_3_01_256,Organoids_Texture_Variance_Hoechst_3_02_256,Organoids_Texture_Variance_Hoechst_3_03_256,Organoids_Texture_Variance_Mito_3_00_256,Organoids_Texture_Variance_Mito_3_01_256,Organoids_Texture_Variance_Mito_3_02_256,Organoids_Texture_Variance_Mito_3_03_256
0,31,0.0,None,None,0,NF0014,0,1,D4,ZS058,...,1.783403,1.785666,507.320199,507.366403,507.537522,507.186298,9.442713,9.441585,9.441276,9.439410
1,11,0.0,None,None,0,NF0014,0,1,C5,ZS052,...,1.947012,1.942769,285.620945,285.249286,285.787073,285.224915,5.527802,5.517586,5.527372,5.499532
2,12,0.0,None,None,0,NF0014,0,2,C5,ZS030,...,7.812409,7.735139,585.425367,586.077236,585.362315,586.266941,81.750646,81.673493,81.765727,81.668038
3,13,0.0,None,None,0,NF0014,0,1,C6,ZS034,...,2.741727,2.767367,263.209426,266.067862,262.416556,262.829797,22.845596,22.953855,22.800617,22.943567
4,14,0.0,None,None,0,NF0014,0,2,C6,ZS054,...,1.265346,1.263935,329.987455,329.667528,329.939192,329.671474,27.272309,27.162311,27.285292,27.181438
